## Creating Acure Container Instance to deploy the model

### Load the workspace using ws_config.json

In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config('./ws_config.json')

In [ ]:
from azureml.core.model import Model

### Load the already registered model

In [ ]:
model = Model(ws, name='20190708-0919_2379-595height')

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

### Create the environment with pip and conda packages along with an entry_script

In [ ]:
env = Environment('deploytocloudenv_ACI')
env.python.conda_dependencies = CondaDependencies.create(conda_packages=['tensorflow'],pip_packages=['azureml-defaults', 'pyntcloud', 'inference_schema', 'opencv-python', 'matplotlib', 'psycopg2-binary', 'tqdm', 'Pillow', 'opencv-python', 'bunch'])
inference_config = InferenceConfig(entry_script="score.py", environment=env)

### To register the created  environment in workspace

In [ ]:
env.register(ws)

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice

### Deploy the model

In [ ]:
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 2)
service = Model.deploy(ws, "aciservice", [model], inference_config, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

### check logs if something goes wrong

In [ ]:
service.get_logs()

### Updating web service

#### get the registered environment and set the inference config(may be after some changes to score.py)

In [ ]:
deploy_env = Environment.get(workspace=ws,name="deploytocloudenv_ACI")
inference_config = InferenceConfig(entry_script="score.py",
                                   environment=deploy_env)

In [ ]:
service_name = 'aciservice'
# Retrieve existing service.
service = Webservice(name=service_name, workspace=ws)

In [ ]:
service.update(inference_config=inference_config)
print(service.state)